In [6]:
from collections import namedtuple, deque
import random
import gym
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import Adam

In [14]:
dq = deque(maxlen=2)

In [20]:
dq.append(3)

In [24]:
dq[-1]

2

In [26]:
??deque

In [22]:
dq.pop()

3

In [23]:
dq

deque([2])

In [13]:
?deque

In [12]:
list(dq)

[]

In [2]:
env = gym.make('CartPole-v0')

In [3]:
action_n = env.action_space.n
observation_shape = env.observation_space.shape

In [5]:
env.reset()
for _ in range(100):
    obs, reward, done, _ = env.step(env.action_space.sample())
    if done:
        print(reward)
        break

1.0


In [7]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    
    
def clip_grads(net, low=-10, high=10):
    """Gradient clipping to the range [low, high]."""
    parameters = [param for param in net.parameters()
                  if param.grad is not None]
    for p in parameters:
        p.grad.data.clamp_(low, high)

In [8]:
memory = ReplayMemory(10000)

In [20]:
net = nn.Sequential(nn.Linear(observation_shape[0],128),nn.ReLU(),nn.Linear(128,action_n))

In [13]:
class DeepQL:
    pass

In [14]:
obs = env.reset()

In [29]:
def select_action(obs, model, thresh=0):
    state = torch.from_numpy(obs).float().unsqueeze(0)
    if np.random.random()>thresh:
        with torch.no_grad():
            probs = model(Variable(state))
            _, action_ = probs.max(1)
        action = action_.data[0]
    else:
        print("Exploration!")
        action = np.random.randint(action_n)
    return action

In [52]:
select_action(obs,net,thresh=0.3)

Exploration!


0

In [55]:
# net.state_dict()

In [ ]:
for episode in range(20):
    obs = env.reset()
    total_reward = 0
    for _ in range(10000): # not exceed 10000 episodes
        action = select_action(obs, net)
        obs, reward, done, _ = trainer.take_action(action, env, render=False)
        total_reward+=reward
        if done:
             break